## Corona-News Scraper from Tagesschau.de

In [30]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup

import datetime
import pandas as pd

In [31]:
todays_date = datetime.datetime.now()
date_stamp = str(todays_date.date())

In [32]:
url = 'https://www.tagesschau.de/'
html = urlopen(url)
soup = BeautifulSoup(html, "html5lib")

### Parse Tagesschau.de homepage for new articles

In [33]:
headlines = []
dachzeilen = []
teasers = []
links = []
texts = []
a = soup.find_all('a')
for item in a:
    if item.find('p', class_='dachzeile'):
        links.append(item.get('href'))
        texts.append(item.get_text())

In [34]:
# check the first text / link
print(texts[0])

print(links[0])



Maßnahmen gegen Corona
Südkorea als Vorbild?
Auf der Suche nach Maßnahmen gegen die Ausbreitung des Coronavirus gilt Südkorea als Vorbild. Dort blieb die Zahl der Infektionen relativ niedrig. Aber auch dort ist die Krise noch nicht vorbei. Von Silvia Stöber.
| mehr


/faktenfinder/corona-suedkorea-111.html


### Check which articles on the homepage are about the Corona virus

In [35]:
# corona, covid19, coronavirus
corona_links = []
corona_texts = []
for i in range(len(texts)):
    if ("corona" in texts[i].lower()) | ("covid" in texts[i].lower()):
        if links[i].startswith("/"):
            corona_links.append(links[i])
            corona_texts.append(texts[i])

In [36]:
print(corona_links)
print("")
print(corona_texts)

['/faktenfinder/corona-suedkorea-111.html', '/ausland/taiwan-corona-101.html', '/ausland/china-corona-fallzahlen-kommission-101.html', '/ausland/trump-analyse-105.html', '/inland/corona-entwicklung-interview-101.html', '/ausland/coronavirus-karten-101.html', '/newsticker/liveblog-corona-dienstag-101.html', '/wirtschaft/heil-kurzarbeit-corona-101.html', '/inland/corona-test-105.html', '/ausland/corona-neuseeland-101.html', '/ausland/usa-kreuzfahrtschiff-zaandam-101.html', '/ausland/corona-usa-111.html', '/inland/corona-bundeswehr-inland-amtshilfe-101.html', '/sport/sportschau/bundesliga-corona-101.html']

['\nMaßnahmen gegen Corona\nSüdkorea als Vorbild?\nAuf der Suche nach Maßnahmen gegen die Ausbreitung des Coronavirus gilt Südkorea als Vorbild. Dort blieb die Zahl der Infektionen relativ niedrig. Aber auch dort ist die Krise noch nicht vorbei. Von Silvia Stöber.\n|\xa0mehr\n\n', '\nTaiwans Corona-Strategie\nWo es Masken von der Krankenkasse gibt\nIn Taiwan gibt es laut Statistik nur 

### Get full article for each news about corona virus

In [37]:
corona_articles = []
for link in corona_links:
    full_link = "https://www.tagesschau.de" + link
    print(full_link)
    html = urlopen(full_link)
    sub_soup = BeautifulSoup(html, "html5lib")
    article = sub_soup.find_all('p', class_='text small')
    paragraphs = []
    for i in range(len(article)):
        paragraphs.append(article[i].get_text())
    # unifying teh paragraphs
    final_article = " ".join(paragraphs)
    corona_articles.append(final_article)
    

https://www.tagesschau.de/faktenfinder/corona-suedkorea-111.html
https://www.tagesschau.de/ausland/taiwan-corona-101.html
https://www.tagesschau.de/ausland/china-corona-fallzahlen-kommission-101.html
https://www.tagesschau.de/ausland/trump-analyse-105.html
https://www.tagesschau.de/inland/corona-entwicklung-interview-101.html
https://www.tagesschau.de/ausland/coronavirus-karten-101.html
https://www.tagesschau.de/newsticker/liveblog-corona-dienstag-101.html
https://www.tagesschau.de/wirtschaft/heil-kurzarbeit-corona-101.html
https://www.tagesschau.de/inland/corona-test-105.html
https://www.tagesschau.de/ausland/corona-neuseeland-101.html
https://www.tagesschau.de/ausland/usa-kreuzfahrtschiff-zaandam-101.html
https://www.tagesschau.de/ausland/corona-usa-111.html
https://www.tagesschau.de/inland/corona-bundeswehr-inland-amtshilfe-101.html
https://www.tagesschau.de/sport/sportschau/bundesliga-corona-101.html


In [38]:
print(corona_articles)

['Auf der Suche nach Maßnahmen gegen die Ausbreitung des Coronavirus gilt Südkorea als Vorbild. Dort blieb die Zahl der Infektionen relativ niedrig. Aber auch dort ist die Krise noch nicht vorbei.  In einem vertraulichen Strategiepapier des Bundesinnenministeriums zum Kampf gegen das Coronavirus wird Südkorea als Vorbild genannt. Mittels Massentests und Isolierung von Erkrankten sei die Ausbreitung des Erregers stark verlangsamt worden, ohne das öffentliche Leben zum Stillstand zu bringen. \n  In der Tat flachte sich die Infektionskurve in Südkorea schnell ab, obwohl es im Februar einen heftigen Ausbruch gab. Bis Dienstag wurden dort lediglich 9786 Menschen positiv auf den Sars-CoV-2-Erreger getestet. Die Zahl der Todesfälle in Verbindung mit dem Virus lag bei 162. In Deutschland sind es fast 67.000 Infizierte und 650 Todesfälle.  Dabei gibt es Ähnlichkeiten zwischen Südkorea und der Bundesrepublik - auch dies ein Grund, das südostasiatische Land zum Vorbild zu nehmen. Mit einer Bevölk

In [39]:
len(corona_articles)

14

### Make into CSV file

In [40]:
columns = ["date", "url", "article", "resort"]
df = pd.DataFrame(columns=columns)
df["url"] = corona_links
df["article"] = corona_articles
df["date"] = todays_date
df["resort"] = df["url"].apply(lambda x: x.split("/")[1])

In [41]:
df.head()

,date,url,article,resort
0,2020-03-31 18:17:41.398236,/faktenfinder/corona-suedkorea-111.html,Auf der Suche nach Maßnahmen gegen die Ausbrei...,faktenfinder
1,2020-03-31 18:17:41.398236,/ausland/taiwan-corona-101.html,In Taiwan gibt es laut Statistik nur etwa 300 ...,ausland
2,2020-03-31 18:17:41.398236,/ausland/china-corona-fallzahlen-kommission-10...,Chinas Coronavirus-Statistiken fallen ungewöhn...,ausland
3,2020-03-31 18:17:41.398236,/ausland/trump-analyse-105.html,Lange hat US-Präsident Trump die Corona-Krise ...,ausland
4,2020-03-31 18:17:41.398236,/inland/corona-entwicklung-interview-101.html,Die aktuellen Maßnahmen verschaffen uns Zeit z...,inland


### Export CSV

In [42]:
df.to_csv("data/" + date_stamp)